In [1]:
import torch
torch.cuda.get_device_name(0)

'Tesla K80'

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import math
use_cuda = True
device = torch.device("cuda:0" if use_cuda else "cpu")

class DAU(nn.Module):
    def __init__(self, input_channels, output_channels, dau_unit=3, max_kernel_size=32):
       super(DAU, self).__init__()
       self.max_kernel_size=max_kernel_size
       self.input_channels=input_channels
       self.output_channels=output_channels
       self.dau_unit = dau_unit
       self.w = nn.Parameter(torch.ones(1, self.output_channels,max_kernel_size,max_kernel_size,requires_grad=True,device=device))
       self.mu1 = nn.Parameter(torch.ones((1, dau_unit),device=device))
       self.mu2 = nn.Parameter(torch.ones((1, dau_unit),device=device))
       self.sigma = 0.5
       self.reset_parameters()
       [X,Y] = np.meshgrid(np.arange(max_kernel_size),np.arange(max_kernel_size))
       X = X.astype(np.float32)
       Y = Y.astype(np.float32)
       self.X = torch.tensor(np.reshape(X,(max_kernel_size*max_kernel_size,1,1,1)) - int(max_kernel_size/2),device=device)
       self.Y = torch.tensor(np.reshape(Y,(max_kernel_size*max_kernel_size,1,1,1)) - int(max_kernel_size/2),device=device)
       self.X = self.X / torch.max(self.X)
       self.Y = self.Y / torch.max(self.Y)

        
    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.w.size(1))
        self.w.data.uniform_(-stdv, stdv)
        stdv = 1. / math.sqrt(self.mu1.size(1))
        self.mu1.data.uniform_(-stdv, stdv)
        stdv = 1. / math.sqrt(self.mu2.size(1))
        self.mu2.data.uniform_(-stdv, stdv)

        
    def forward(self, inputs):
        # print(torch.mean(self.mu1))
        # print(torch.mean(self.mu2))
        gaussian_kernels_norm=[]
 
        for i in range(self.dau_unit):
            gaussian_kernel = torch.exp(-1*(torch.pow(self.X-self.mu1[0][i],2)+torch.pow(self.Y-self.mu2[0][i],2))/(2.0*self.sigma**2))
            gauss_kernel_sum = torch.sum(gaussian_kernel, dim =0, keepdim=True)
            gauss_kernel_norm = torch.div(gaussian_kernel, gauss_kernel_sum)
            gaussian_kernels_norm.append(gauss_kernel_norm)

        gauss_kernel_norm = torch.cat(gaussian_kernels_norm, dim=1)

        gauss_kernel_norm = torch.sum(gauss_kernel_norm,dim =1, keepdim=True,)

        gauss_kernel_norm = torch.reshape(gauss_kernel_norm, (1, 1, self.max_kernel_size, self.max_kernel_size,))
        gauss_kernel_norm = gauss_kernel_norm.repeat(1,self.input_channels,1,1)
        # print(gauss_kernel_norm.size())
        output = F.conv2d(inputs,gauss_kernel_norm,padding='same')
        # print(output.size())
        output = output.mul(self.w)

        return output

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 16

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=1)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=1)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        device = torch.device('cuda:0')
        self.DAUs=nn.ModuleList([DAU(3,12,3).to(device), DAU(12,6,6).to(device)])
        self.fc1 = nn.Linear(6144, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 10)

    def forward(self, x):
        for i, l in enumerate(self.DAUs):
            x = self.DAUs[i](x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x



In [5]:
import torch.optim as optim
from torchsummary import summary

net = Net()
net.to(torch.device('cuda:0'))
criterion = nn.CrossEntropyLoss()

optimizer = optim.SGD([
    {'params': net.DAUs[0].mu1, 'lr': 0.04},
    {'params': net.DAUs[0].mu2, 'lr': 0.04},
    {'params': net.DAUs[1].mu1, 'lr': 0.04},
    {'params': net.DAUs[1].mu2, 'lr': 0.04},
    {'params': net.DAUs[0].w, 'lr': 0.04},
    {'params': net.DAUs[1].w, 'lr': 0.04},
    {'params': net.fc1.weight},
    {'params': net.fc2.weight}], lr=0.01, momentum=0.9,weight_decay=0.0005)


#summary(net,(3,32,32))

In [ ]:
for epoch in range(100):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        labels.to(device)
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: UserWarning: Using padding='same' with even kernel lengths and odd dilation may require a zero-padded copy of the input be created (Triggered internally at  ../aten/src/ATen/native/Convolution.cpp:647.)


[1,  2000] loss: 2.148
[2,  2000] loss: 2.065
[3,  2000] loss: 2.040
[4,  2000] loss: 2.025
[5,  2000] loss: 2.007
[6,  2000] loss: 2.004
[7,  2000] loss: 2.000
[8,  2000] loss: 1.995
[9,  2000] loss: 1.996
[10,  2000] loss: 1.993
[11,  2000] loss: 1.990
[12,  2000] loss: 1.989
[13,  2000] loss: 1.987
[14,  2000] loss: 1.988
[15,  2000] loss: 1.984
[16,  2000] loss: 1.982
[17,  2000] loss: 1.974
[18,  2000] loss: 1.971
[19,  2000] loss: 1.973
[20,  2000] loss: 1.963
[21,  2000] loss: 1.966
[22,  2000] loss: 1.964
[23,  2000] loss: 1.961
[24,  2000] loss: 1.959
[25,  2000] loss: 1.955
[26,  2000] loss: 1.948
[27,  2000] loss: 1.947
[28,  2000] loss: 1.947
[29,  2000] loss: 1.947
[30,  2000] loss: 1.938
[31,  2000] loss: 1.941
[32,  2000] loss: 1.937
[33,  2000] loss: 1.939
[34,  2000] loss: 1.935
[35,  2000] loss: 1.929
[36,  2000] loss: 1.932
[37,  2000] loss: 1.920
[38,  2000] loss: 1.919
[39,  2000] loss: 1.914
[40,  2000] loss: 1.920


In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 30 %
